In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
dataset = pd.read_csv("D:/assets/data/ml-latest-small/ml-latest-small/ratings.csv")
dataset

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [3]:
arr = np.array(dataset, dtype='int')

nb_users = int(max(arr[:, 0]))
nb_movies = len(dataset.movieId.unique())
print(nb_movies, nb_users)

9724 610


In [4]:
pivot_table = dataset.pivot_table(index = ["userId"], columns = ["movieId"], values = "rating", fill_value=0)
pivot_table

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0,0.0,4.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0


In [5]:
data = pivot_table.to_numpy()
data.shape

(610, 9724)

In [6]:
data = torch.FloatTensor(data)

In [7]:
class SAE(nn.Module):

    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 8)
        self.fc3 = nn.Linear(8, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Tanh()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x


sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the gpu")
else:
    device = torch.device("cpu")
    print("running on the cpu")

sae.to(device)

running on the gpu


SAE(
  (fc1): Linear(in_features=9724, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=9724, bias=True)
  (activation): Tanh()
)

In [9]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for user_id in range(nb_users):
        input = Variable(data[user_id]).unsqueeze(0).to(device)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            target = target.to(device)
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data.cpu() * mean_corrector)
            s += 1
            optimizer.step()
    print('epoch: ' + str(epoch) + '  loss: ' + str(train_loss / s))

epoch: 1  loss: tensor(3.7252)
epoch: 2  loss: tensor(3.4019)
epoch: 3  loss: tensor(2.4524)
epoch: 4  loss: tensor(1.9948)
epoch: 5  loss: tensor(1.7605)
epoch: 6  loss: tensor(1.6154)
epoch: 7  loss: tensor(1.5159)
epoch: 8  loss: tensor(1.4429)
epoch: 9  loss: tensor(1.3865)
epoch: 10  loss: tensor(1.3419)


In [10]:
torch.save(sae.state_dict(), 'models/sae_10.pt')